In [1]:
from pathlib import Path
import json
import os
from pprint import pprint
from rdflib import RDF, BNode, Graph, Literal, URIRef, Namespace
from scipy.optimize import linear_sum_assignment

from buildingmotif.namespaces import BRICK, BACNET
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
from buildingmotif.template_finder.template_finder import calculate_template_cost, get_templates_param_classes, Token
from buildingmotif.template_finder.calculate_cost import calculate_cost, create_cost_matrix



PROJECT_DIR = Path(os.getcwd()).resolve().parent

In [2]:
bm = BuildingMOTIF("sqlite://")
BLDG = Namespace("urn:building/")
Library.load(ontology_graph="https://github.com/BrickSchema/Brick/releases/download/nightly/Brick.ttl")
ground_truth = Library.load(ontology_graph=str(PROJECT_DIR / "gabes_points" / "sprucehall.ttl"))
equiment_templates = Library.load(directory=str(PROJECT_DIR / "gabes_points" / "equiment_templates"))
templates = equiment_templates.get_templates()

templates

2023-09-20 13:11:46,972 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6897
2023-09-20 13:11:46,972 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6899
2023-09-20 13:11:46,973 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6901
2023-09-20 13:11:46,973 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6903
2023-09-20 13:11:46,973 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6905
2023-09-20 13:11:46,974 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2023-09-20 13:11:46,974 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6847
2023-09-20 13:11:46,974 | root |  WARNING: Warning: could not find dependee nbbb0404f1f4e43f1b7139c29e69b1b4bb6849
2023-09-20 13:11:46,974 | root |  WARNING: Warning: could not find 

[Template(_id=1328, _name='fcu', body=<Graph identifier=b4600991-ae93-4ed0-ba52-e5ed241a949f (<class 'rdflib.graph.Graph'>)>, optional_args=['occ_cmd', 'occ_status', 'occ_clg_sp', 'unocc_clg_sp', 'occ_htg_sp', 'unocc_htg_sp'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x124d9f4d0>),
 Template(_id=1329, _name='room', body=<Graph identifier=c5afca22-4e5a-4e11-9653-5bf233352c60 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x124d9f4d0>),
 Template(_id=1330, _name='heating_coil', body=<Graph identifier=82e4e60b-c19d-4537-87e6-a02fc2872de7 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x124d9f4d0>),
 Template(_id=1331, _name='cooling_coil', body=<Graph identifier=3d8e9e25-ddbd-43cb-8861-6fed1c0a34e8 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingM

In [3]:
with open(PROJECT_DIR / "gabes_points" / "tokens.json") as f:
    labels_and_tokens = json.load(f)

pprint(labels_and_tokens[0])

{'label': ':BuildingName_02:FCU503_ChwVlvPos',
 'tokens': [{'identifier': 'BuildingName',
             'type': 'https://brickschema.org/schema/Brick#Entity'},
            {'identifier': '02',
             'type': 'https://brickschema.org/schema/Brick#Air_Handling_Unit'},
            {'identifier': '503',
             'type': 'https://brickschema.org/schema/Brick#Fan_Coil_Unit'},
            {'identifier': ':BuildingName_02:FCU503_ChwVlvPos',
             'type': 'https://brickschema.org/schema/Brick#Chilled_Water_Valve'}]}


In [4]:
def scalar_cost(cost_dict):
    return cost['edge_cost'] + cost['params_dropped'] + cost['tokens_dropped']

In [10]:
example = [URIRef(x["type"]) for x in labels_and_tokens[0]["tokens"]]
best_templates = {}
mappings = {}

for label_and_tokens in labels_and_tokens[:10]:
    label, tokens = label_and_tokens["label"], label_and_tokens["tokens"]
    print(f"\n=== {label} ===")
    tokens = [Token(classname=URIRef(t['type']), identifier=t['identifier']) for t in tokens]
    print(f"{tokens=}")

    lowest_cost = float('inf')
    best_template = None
    best_mapping = None
 
    for template in templates:
        template = template.inline_dependencies()
        print(f"--- {template.name} ---")
        cost = calculate_template_cost(tokens, template, verbose=False)
        print(scalar_cost(cost), cost)
        if scalar_cost(cost) < lowest_cost:
            print(f"{template.name=}")
            print(f"{cost=}")
            lowest_cost = scalar_cost(cost)
            best_template = template
            best_mapping = cost['mapping']
    best_templates[label] = best_template.in_memory_copy()
    mappings[label] = best_mapping.copy()

    print(best_mapping)


=== :BuildingName_02:FCU503_ChwVlvPos ===
tokens=[Token(identifier='BuildingName', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Entity')), Token(identifier='02', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Air_Handling_Unit')), Token(identifier='503', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Fan_Coil_Unit')), Token(identifier=':BuildingName_02:FCU503_ChwVlvPos', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Chilled_Water_Valve'))]
--- fcu ---

kept edges:
Fan_Coil_Unit <- 0.0 -> Fan_Coil_Unit
Chilled_Water_Valve <- 0.0 -> Chilled_Water_Valve
18.0 {'mapping': {Param(name='name', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Fan_Coil_Unit')): Token(identifier='503', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Fan_Coil_Unit')), Param(name='chw_coil-chw_vlv', classname=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Chilled_Water_Valve')): Token(

In [11]:
for label, template in best_templates.items():
    print(label)
    print(template.name)
    bindings = {p.name: BLDG[t.identifier] for p, t in mappings[label].items()}
    print(bindings)
    _, g = template.evaluate(bindings).fill(BLDG)
    print(g.serialize())

:BuildingName_02:FCU503_ChwVlvPos
cooling_coil
{'chw_vlv': rdflib.term.URIRef('urn:building/:BuildingName_02:FCU503_ChwVlvPos')}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/name_ae489915> a brick:Cooling_Coil ;
    brick:hasPart <urn:building/:BuildingName_02:FCU503_ChwVlvPos> .

<urn:building/:BuildingName_02:FCU503_ChwVlvPos> a brick:Chilled_Water_Valve ;
    brick:hasPoint <urn:building/position_17fcc2a5> .

<urn:building/position_17fcc2a5> a brick:Valve_Position_Command .


:BuildingName_01:FCU336_OccHtgSptFnl
fcu
{'name': rdflib.term.URIRef('urn:building/336')}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/336> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:building/zone_132d0272> ;
    brick:hasPart <urn:building/chw_coil_ab1101fc>,
        <urn:building/hw_coil_1ad4176a> ;
    brick:hasPoint <urn:building/supply_temp_9d45d1b4> .

<urn:building/chw_coil-chw_vlv_563ccd30> a brick:Chilled_Water_Valve ;
    brick:hasPoint <urn:buil

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:372: UserWarning: Parameters "name, position" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:372: UserWarning: Parameters "room, room-relative_humidity, occ_status, occ_clg_sp, occ_htg_sp, chw_coil-chw_vlv, occ_cmd, hw_coil-position, chw_coil-position, room-temp, chw_coil, hw_coil-hw_vlv, zone, hw_coil, unocc_htg_sp, supply_temp, unocc_clg_sp" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:372: UserWarning: Parameters "room, room-relative_humidity, occ_clg_sp, occ_htg_sp, chw_coil-chw_vlv, occ_cmd, hw_coil-position, chw_coil-position, room-temp, chw_coil, hw_coil-hw_vlv, zone, hw_coil, unocc_htg_sp, supply_temp, unocc_clg_sp" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:372: User

AssertionError: 